<a href="https://colab.research.google.com/github/deep-diver/auto-data-fountain/blob/main/notebooks/pilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai
!pip install pyyaml

In [7]:
GEMINI_API_KEY="..."

In [1]:
%%writefile counsel.yaml

base_person: &base
  name: John Doe
  age: 30

counselor:
  <<: *base
  specialization: Psychology

counselee:
  <<: *base
  primary_concern: Anxiety

Writing counsel.yaml


In [3]:
import yaml

yaml_file = "counsel.yaml"

with open(yaml_file, 'r') as stream:
    try:
        data = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)  # Handle potential parsing errors

# Print the parsed data
print(data)

{'base_person': {'name': 'John Doe', 'age': 30}, 'counselor': {'name': 'John Doe', 'age': 30, 'specialization': 'Psychology'}, 'counselee': {'name': 'John Doe', 'age': 30, 'primary_concern': 'Anxiety'}}


In [28]:
def determine_model_name(given_image=None):
  if given_image is None:
    return "gemini-pro"
  else:
    return "gemini-pro-vision"

def construct_image_part(given_image):
  return {
    "mime_type": "image/jpeg",
    "data": given_image
  }

def call_gemini(prompt="", API_KEY=None, given_text=None, given_image=None, generation_config=None, safety_settings=None):
  import google.generativeai as genai
  genai.configure(api_key=API_KEY)

  if generation_config is None:
    generation_config = {
      "temperature": 0.9,
      "top_p": 1,
      "top_k": 32,
      "max_output_tokens": 8192,
    }

  if safety_settings is None:
    safety_settings = [
      {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_ONLY_HIGH"
      },
      {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_ONLY_HIGH"
      },
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_ONLY_HIGH"
      },
      {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_ONLY_HIGH"
      },
    ]

  model_name = determine_model_name(given_image)
  model = genai.GenerativeModel(model_name=model_name,
                                generation_config=generation_config,
                                safety_settings=safety_settings)

  USER_PROMPT = prompt
  if given_text is not None:
    USER_PROMPT = f"""
------------------------------------------------
{given_text}
"""
  prompt_parts = [USER_PROMPT]
  if given_image is not None:
    prompt_parts.append(construct_image_part(given_image))

  response = model.generate_content(prompt_parts)
  return response.text

In [58]:
prompt = """
The below erDiagram describes the basic setup of a certain scene.

Generate possible conversations between a user and an assistant.
The conversations should sound natural and logical.
The conversations should be occured without exposuring the underlying information of the erDiagram.

The user should play the role of "CUSTOMER" appeared in the erDiagram.
The user should start conversations focused on its role, "CUSTOMER".
Based on the words that the user say, the assistant gives appropriate, detailed, and long answers.

The generated conversations are recorded in a valid JSON as
{"conversations":[{"user": text, "assistant": text},...]}.
------------------------------
erDiagram
    CUSTOMER {
        string name
        string custNumber
        string sector
    }
    ORDER {
        int orderNumber
        string deliveryAddress
    }
    LINE-ITEM {
        string productCode
        int quantity
        float pricePerUnit
    }
    CUSTOMER ||--o{ ORDER : places
    ORDER ||--|{ LINE-ITEM : contains
"""

In [59]:
test_json = call_gemini(
    prompt=prompt,
    API_KEY=GEMINI_API_KEY
)

In [60]:
test_json

'``` JSON\n{\n "conversations": [\n  {\n   "user": "Hi there, I am new to online shopping. Can you show me how to order?",\n   "assistant": "Of course, here are the steps to place a new order: First, you\'ll need to create a customer account. Once you are registered, you can browse our wide selection of products. When you find an item you wish to purchase, simply add it to your cart. Once your shopping is complete, go to your cart and follow the checkout instructions."\n  },\n  {\n   "user": "What payment methods do you accept?",\n   "assistant": "We offer a range of safe and secure payment options, including credit and debit cards, online banking, and PayPal. We value the security of your financial information, so we make sure to utilize cutting-edge encryption technology."\n  },\n  {\n   "user": "How much does shipping cost?",\n   "assistant": "Shipping costs depend on the weight and size of your order, as well as your selected shipping method. We offer regular shipping, which usuall

In [62]:
import json

def find_json_code_snippet(raw_code_snippet):
	json_parsed_string = None

	json_start_index = raw_code_snippet.find('{')
	json_end_index = raw_code_snippet.rfind('}')

	if json_start_index >= 0 and json_end_index >= 0:
		json_code_snippet = raw_code_snippet[json_start_index:json_end_index+1]
		try:
			json_parsed_string = json.loads(json_code_snippet, strict=False)
		except:
			raise ValueError('failed to parse string into JSON format')
	else:
		raise ValueError('No JSON code snippet found in string.')

	return json_parsed_string

def parse_first_json_code_snippet(code_snippet):
	json_parsed_string = None

	if isinstance(code_snippet, list):
		for code_snippet_piece in code_snippet:
			try:
				json_parsed_string = find_json_code_snippet(code_snippet_piece)
				return json_parsed_string
			except:
				pass
	else:
		try:
			json_parsed_string = find_json_code_snippet(code_snippet)
		except Exception as e:
			print(e)
			raise ValueError()

	return json_parsed_string

In [63]:
test_json = parse_first_json_code_snippet(test_json)

In [65]:
test_json

{'conversations': [{'user': 'Hi there, I am new to online shopping. Can you show me how to order?',
   'assistant': "Of course, here are the steps to place a new order: First, you'll need to create a customer account. Once you are registered, you can browse our wide selection of products. When you find an item you wish to purchase, simply add it to your cart. Once your shopping is complete, go to your cart and follow the checkout instructions."},
  {'user': 'What payment methods do you accept?',
   'assistant': 'We offer a range of safe and secure payment options, including credit and debit cards, online banking, and PayPal. We value the security of your financial information, so we make sure to utilize cutting-edge encryption technology.'},
  {'user': 'How much does shipping cost?',
   'assistant': 'Shipping costs depend on the weight and size of your order, as well as your selected shipping method. We offer regular shipping, which usually takes around 3-5 business days, and express s

In [66]:
prompt = """
The below erDiagram describes the basic setup of a certain scene.

Generate possible conversations between a user and an assistant.
The conversations should sound natural and logical.
The conversations should be occured without exposuring the underlying information of the erDiagram.

The user should play the role of "COUNSELEE" appeared in the erDiagram.
The user should start conversations focused on its role, "COUNSELEE".
The assistant should play the role of "COUNSELOR" appeared in the erDiagram.
The assistant should start conversations focused on its role, "COUNSELOR".
Based on the words that the user say, the assistant gives appropriate, detailed, and long answers.

The generated conversations are recorded in a valid JSON as
{"conversations":[{"user": text, "assistant": text},...]}.
------------------------------
erDiagram
    COUNSELOR ||--|{ COUNSELEE : "provides counseling to"

    %% Comments for relationship attributes
    %% Start date: 2024-02-14
    %% Frequency: Weekly
"""

In [70]:
test_json = call_gemini(
    prompt=prompt,
    API_KEY=GEMINI_API_KEY
)

In [71]:
test = parse_first_json_code_snippet(test_json)

In [72]:
test

{'conversations': [{'user': "Hi, I'm feeling really down lately. I'm not sure what to do.",
   'assistant': "I'm sorry to hear that you're feeling down. How long have you been feeling this way?"},
  {'user': "I've been feeling down for a few weeks now. I'm not sure what caused it.",
   'assistant': "I understand. Sometimes it's hard to pinpoint the exact cause of our feelings. It's important to know that you're not alone and that there is help available."},
  {'user': "I know, but I'm still struggling. I don't know where to turn.",
   'assistant': "I can help you with that. I'm a counselor, and I'm here to provide support and guidance. We can work together to figure out what's causing your feelings and develop strategies to cope with them."},
  {'user': 'Thank you. I appreciate your help.',
   'assistant': "You're welcome. I'm here to listen and help you through this."},
  {'user': "I'm not sure what to talk about.",
   'assistant': "That's perfectly fine. We can start by talking about